In [1]:
from pathlib import Path
import pandas as pd
import SimpleITK as sitk
import pydicom
import matplotlib.pyplot as plt

This notebook provides an example of converting a metaheader dataset to DICOM.

Why?

Metaheader datasets are simple raw "brick of pixels" data formats with a simple header file with pixel size and orientation information. This header can be either saved as a separate file `.mhd` file by convention, or prepended to the dataset. While simple and useful for research purposes, meta header files are not commonly used in clinical images, DICOM files are the standard there. DICOM files are more complicated but have much more information that can be encoded, which leads to the complications. This notebook does not teach you much about what all of the DICOM header info means, but it does give hands on examples of what elements are changd and how

## Organize Dataset

In [2]:
!ls /gpfs_projects/brandon.nelson/DLIR_Ped_Generalizability/geometric_phantom_studies/main/geometric/CCT189/monochromatic/diameter112mm/

geometry_info.csv     I0_0075000	    I0_0255000_processed
I0_0010000	      I0_0075000_processed  I0_0300000
I0_0010000_processed  I0_0085000	    I0_0300000_processed
I0_0025000	      I0_0085000_processed  image_geometry.png
I0_0025000_processed  I0_0100000	    image_info.csv
I0_0030000	      I0_0100000_processed  noise_free_bkg.raw
I0_0030000_processed  I0_0120000	    noise_free_disk.raw
I0_0040000	      I0_0120000_processed  phantom_info_mm.csv
I0_0040000_processed  I0_0165000	    phantom_info_pix_idx.csv
I0_0055000	      I0_0165000_processed  true_bkg.raw
I0_0055000_processed  I0_0210000	    true_disk.raw
I0_0070000	      I0_0210000_processed
I0_0070000_processed  I0_0255000


In [3]:
orig_dir = Path('/gpfs_projects/brandon.nelson/DLIR_Ped_Generalizability/geometric_phantom_studies/main_noise_free/geometric')
dst_dir = Path('/gpfs_projects/brandon.nelson/RSTs/pediatricIQphantoms')
noise_free_imgs = list(orig_dir.rglob('*/noise_free*.raw'))

In [4]:
from datetime import datetime
import numpy as np

phantom_dict = {'CTP404': 'CTP404', 'MITA-LCD': 'CCT189', 'uniform': 'CCT189'}

def get_ground_truth(fname):
    fname = Path(fname)
    if fname.stem.startswith('signal'):
        gt_file = 'noise_free.mhd'
        return Path(fname).parents[2] / gt_file
    if fname.stem.startswith('ACR464'):
        gt_file = 'true.mhd'
        return Path(fname).parents[3] / gt_file
    else:
        gt_file = 'true.mhd'
        return Path(fname).parents[2] / gt_file
    
def write_series_to_dicom(series, save_dir):
    # load default dicom files
    series = series.copy()
    fpath = pydicom.data.get_testdata_file("CT_small.dcm")
    ds = pydicom.dcmread(fpath).copy()
    # update meta info
    ds.Manufacturer = 'Siemens (simulated)'
    ds.ManufacturerModelName = 'Definition AS+ (simulated)'

    time = datetime.now()
    ds.InstanceCreationDate = time.strftime('%Y%m%d')
    ds.InstanceCreationTime = time.strftime('%H%M%S')
    ds.InstitutionName = 'FDA/CDRH/OSEL/DIDSR'
    ds.StudyDate = ds.InstanceCreationDate
    ds.StudyTime = ds.InstanceCreationTime
    age = series['age [year]']
    if not age >= 1:
        age = 0
    ds.PatientName = series.Name
    ds.SeriesNumber = str(1)

    ds.PatientAge = f'{int(age):03d}Y'
    ds.PatientID = f'{int(series.patientid):03d}'
    del(ds.PatientWeight)
    del(ds.ContrastBolusRoute)
    del(ds.ContrastBolusAgent)
    ds.ImageComments = f"effctive diameter [cm]: {series['effective diameter [cm]']}"
    ds.ScanOptions = 'AXIAL MODE'
    ds.ReconstructionDiameter = series['FOV [cm]'] * 10
    ds.ConvolutionKernel ='fbp D45'
    ds.Exposure = series['Dose [%]']

    # load image data
    img = sitk.ReadImage(series.file)
    ds.StudyDescription = f"{series['Dose [%]']}% dose " + series.Name + " " + ds.ConvolutionKernel

    if series['series'] == 'ground truth':
        img = sitk.ReadImage(get_ground_truth(series.file))
        vol = sitk.GetArrayFromImage(img)
        img = sitk.GetImageFromArray(vol[None])
        fov = series['FOV [cm]']*10
        img.SetSpacing((fov/512, fov/512, fov/512))
        series.Name += '_groundtruth'
        ds.StudyDescription = series.Name
    
    elif series['series'] == 'noise free':
        series.Name += '_noisefree'
        ds.StudyDescription = series.Name + " " + ds.ConvolutionKernel

        phantom_str = phantom_dict[phantom]
        diameter = int(series['effective diameter [cm]']*10)
        diam_filter = np.array([str(diameter) in str(o) for o in noise_free_imgs])
        phantom_filter = np.array([str(phantom_str) in str(o) for o in noise_free_imgs])
        select_noise_free =  np.array(noise_free_imgs)[diam_filter & phantom_filter]
        # print(phantom_str, diameter)
        if phantom == 'uniform':
            select_noise_free = select_noise_free[[o.stem.endswith('bkg') for o in select_noise_free]]
        if phantom == 'MITA-LCD':
            select_noise_free = select_noise_free[[o.stem.endswith('disk') for o in select_noise_free]]
        
        # print(select_noise_free)
        assert(len(select_noise_free) == 1)
        noise_free_file = select_noise_free[0]
        vol = np.fromfile(noise_free_file, dtype='int16').reshape((512, 512))
        img = sitk.GetImageFromArray(vol[None]) - 1000
    
    fov = series['FOV [cm]']*10
    img.SetSpacing((fov/512, fov/512, fov/512))
    ds.SeriesDescription = ds.StudyDescription

    vol = sitk.GetArrayFromImage(img)
    if vol.ndim == 2: vol = vol[None]
    
    ds.Rows, ds.Columns = img.GetHeight(), img.GetWidth()
    ds.SliceThickness = img.GetSpacing()[-1]
    ds.SpacingBetweenSlices = ds.SliceThickness
    ds.DistanceSourceToDetector = 1085.6
    ds.DistanceSourceToPatient = 595
    ds.PixelSpacing = list(img.GetSpacing()[:2])

    ds.KVP = 120
    ds.StudyID = str(series['studyid'])
    # series instance uid unique for each series
    end = ds.SeriesInstanceUID.split('.')[-1]
    new_end = str(int(end) + series['studyid'])
    ds.SeriesInstanceUID = ds.SeriesInstanceUID.replace(end, new_end)
    
    # study instance uid unique for each series
    end = ds.StudyInstanceUID.split('.')[-1]
    new_end = str(int(end) + series['studyid'])
    ds.StudyInstanceUID = ds.StudyInstanceUID.replace(end, new_end)
    ds.AcquisitionNumber = series['studyid']

    save_dir = Path(save_dir)
    save_dir.mkdir(exist_ok=True, parents=True)
    
    # saveout slices as individual dicom files
    fnames = []
    for slice_idx, array_slice in enumerate(vol):
        ds.InstanceNumber = slice_idx + 1 # image number
        # SOP instance UID changes every slice
        end = ds.SOPInstanceUID.split('.')[-1]
        new_end = str(int(end) + slice_idx + series.name)
        ds.SOPInstanceUID = ds.SOPInstanceUID.replace(end, new_end)
        # MediaStorageSOPInstanceUID changes every slice
        end = ds.file_meta.MediaStorageSOPInstanceUID.split('.')[-1]
        new_end = str(int(end) + slice_idx + series.name)
        ds.file_meta.MediaStorageSOPInstanceUID = ds.file_meta.MediaStorageSOPInstanceUID.replace(end, new_end)
        # slice location and image position changes every slice
        ds.SliceLocation = -img.GetDepth()//2*ds.SliceThickness + slice_idx*ds.SliceThickness
        ds.ImagePositionPatient[-1] = ds.SliceLocation
        ds.ImagePositionPatient[0] = -ds.Rows//2*ds.PixelSpacing[0]
        ds.ImagePositionPatient[1] = -ds.Columns//2*ds.PixelSpacing[1]
        ds.ImagePositionPatient[2] = ds.SliceLocation
        ds.PixelData = array_slice.astype('int16') - int(ds.RescaleIntercept)
        if series['series'] == 'simulation':
            fname = save_dir / f"{series.Name.replace('.', '').replace(' cm', 'mm').replace(' ', '_')}_{slice_idx:03d}.dcm"
        else:
            fname = save_dir.parents[1] / f"{series.Name.replace('.', '').replace(' cm', 'mm').replace(' ', '_')}.dcm"
        fnames.append(fname)
        pydicom.write_file(fname, ds)
    return fnames

In [5]:
from tqdm import tqdm
from shutil import rmtree
from utils import pediatric_subgroup, subgroup_to_age

base_dir = Path('/gpfs_projects/brandon.nelson/PediatricCTSizeDataAugmentation')
dst_dir = Path('/gpfs_projects/brandon.nelson/RSTs/pediatricIQphantoms')

overwrite=True
if overwrite:
    
    meta = pd.read_csv(base_dir / 'metadata.csv')
    meta.loc[meta.phantom == 'CTP404', 'Name']=meta[meta.phantom == 'CTP404']['Name'].apply(lambda o: o.replace('mm', 'cm'))
    meta = meta[meta.phantom.isin(['MITA-LCD', 'uniform', 'CTP404']) & meta.recon.isin(['fbp']) & meta['Dose [%]'].isin([25, 100])]
    meta['kernel']='D45'
    meta['scanner']='Siemens Somatom Definition'
    meta.pop('Code #')
    meta.pop('ethnicity')
    meta.pop('weight percentile')
    meta.pop('height [cm]')
    meta.pop('weight [kg]')
    meta.pop('BMI')
    meta.pop('gender')
    meta['pediatric subgroup'] = meta['effective diameter [cm]'].apply(pediatric_subgroup)
    meta['age [year]'] = meta['pediatric subgroup'].apply(subgroup_to_age)
    meta.sort_values(by=['phantom', 'effective diameter [cm]', 'Dose [%]'], inplace=True)
    meta = meta[['Name', 'effective diameter [cm]', 'age [year]', 'pediatric subgroup', 'phantom', 'scanner', 'Dose [%]', 'recon', 'kernel', 'FOV [cm]', 'file']]
    meta.reset_index(drop=True, inplace=True)
    for idx, name in enumerate(meta['Name'].unique()): meta.loc[meta['Name']==name, 'patientid'] = idx
    meta['studyid'] = meta.index
    
    if dst_dir.exists():
        rmtree(dst_dir)

    df_list = []
    # meta = meta.iloc[:2]
    studyid = 0
    for idx, series in tqdm(meta.iterrows(), total=len(meta)):
        phantom = series.phantom
        diameter = series['effective diameter [cm]']
        dose = series['Dose [%]']
        series['series']='simulation'
        recon = series.recon
        save_subdir = dst_dir / phantom / f'diameter_{int(10*diameter):03d}mm' / f'dose_{dose:03d}' / recon
        series['studyid'] = studyid
        studyid += 1
        fnames = write_series_to_dicom(series, save_subdir)
        temp_df = pd.concat(len(fnames)*[pd.DataFrame(series).T], ignore_index=True)
        temp_df['repeat']=list(map(lambda o: int(o.stem.split('_')[-1]), fnames)) # get repeat number from filename
        temp_df['file']=fnames
        df_list.append(temp_df)
        # add ground truth
        if series['Dose [%]'] == 100:
            series['series']='ground truth'
            series['studyid'] = studyid
            studyid += 1
            fnames = write_series_to_dicom(series, save_subdir)
            temp_df = pd.concat(len(fnames)*[pd.DataFrame(series).T], ignore_index=True)
            temp_df['repeat']=0
            temp_df['file']=fnames
            df_list.append(temp_df)
            # add noise free
            series['series']='noise free'
            series['studyid'] = studyid
            studyid += 1
            fnames = write_series_to_dicom(series, save_subdir)
            temp_df = pd.concat(len(fnames)*[pd.DataFrame(series).T], ignore_index=True)
            temp_df['repeat']=0
            temp_df['file']=fnames
            df_list.append(temp_df)
        
    out_meta = pd.concat(df_list, ignore_index=True)
    out_meta.loc[out_meta['series'] == 'ground truth', 'recon'] = 'ground truth'
    out_meta.loc[out_meta['series'] == 'noise free', 'recon'] = 'noise free'
    out_meta.loc[out_meta['series'] == 'ground truth', 'Dose [%]'] = None
    out_meta.loc[out_meta['series'] == 'noise free', 'Dose [%]'] = None    
    out_meta.loc[out_meta['series'] == 'ground truth', 'kernel'] = None
    out_meta.loc[out_meta['series'] == 'noise free', 'kernel'] = None
    out_meta[~out_meta.file.duplicated()]
    out_meta = out_meta[~out_meta.duplicated()]
    out_meta.file = out_meta.file.apply(lambda o: Path(o).relative_to(dst_dir))
    out_meta.to_csv(dst_dir/'metadata.csv', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████| 48/48 [04:35<00:00,  5.75s/it]


!cd /gpfs_projects/brandon.nelson/RSTs & zip pediatricIQphantoms.zip pediatricIQphantoms/ -r -q